In [ ]:
#https://github.com/sudheernaidu53/Machine-learning-Deep-learning-projects

!pip install PyPDF2
!pip install docx2txt
!pip install datasets evaluate transformers[sentencepiece]
!pip install language_tool_python

In [ ]:
import time
start = time.time()

## **Import Libraries**

In [ ]:
import numpy as np
import re
import random
import json
import requests
import docx2txt
import PyPDF2
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from datetime import datetime
from nltk import sent_tokenize
from nltk import pos_tag, word_tokenize, sent_tokenize
from collections import defaultdict
from requests.api import request

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## **Import File API**

In [ ]:
temp_id = "new id"
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/632b087ba62315a24c028622")
resp= request.json()
file_path=(resp["data"]["file_path"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])

In [ ]:
if file_type == "txt":
    import urllib.request
    response = urllib.request.urlopen(file_path)
    html = response.read()
    text=html.decode('utf8')
elif file_type=="pdf":
    url = file_path
    response = requests.get(url)
    my_raw_data = response.content

    with open("my_pdf.pdf", 'wb') as my_data:
        my_data.write(my_raw_data)

    open_pdf_file = open("my_pdf.pdf", 'rb')
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    if read_pdf.isEncrypted:
        read_pdf.decrypt("")
        text = read_pdf.getPage(0).extractText()
    else:
        text = read_pdf.getPage(0).extractText()
elif file_type == "docx":
    url = file_path
    response = requests.get(url)
    my_raw_data = response.content

    with open("my_doc.txt", "wb") as text_file:
        text_file.write(my_raw_data)

    open_docx_file = open("my_doc.txt", 'rb')

    text = docx2txt.process(open_docx_file)
else:
    print("Invalid File Type")


In [ ]:
if len(text)==0:
    print("file is empty.")
else:
    if not re.match(r'!@#$%^&"*()-+?_=,<>/\'', text) :
        print("valid")
    elif text=="'":
        print("invalid")
    else:
        print('Invalid file')

valid


## **Data Cleaning And Data Preprossing**

**Split Text Into Sentences**

In [ ]:
sentences = sent_tokenize(text)
text_to_sentence= np.array(sentences)

**Split Sentences Into Words**

In [ ]:
sentence_to_words=[]
for i in text_to_sentence[0:]:
    sentence_to_words.extend(i.split()) 

In [ ]:
tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]

**Remove Punctuation**

In [ ]:
remove_punctuation = string.punctuation
remove_punctuation = remove_punctuation.replace(".", "")
pattern = r"[{}]".format(re.escape(remove_punctuation))
table = str.maketrans('', '', pattern)
stripped = [w.translate(table) for w in tokens]

**Join Words Into Sentences**

In [ ]:
words_to_sentense=' '.join(stripped)

**Data Cleaning**

In [ ]:
def clean_and_tag(paragraph):
    para = re.sub(r'[^a-zA-Z0-9 .-]','',paragraph)
    tags_dict_sentences = defaultdict(list)
    for sentence in sent_tokenize(para):
        temp_tags = pos_tag(word_tokenize(sentence))
        tags_dict = defaultdict(list)
        tags_dict_sentences[sentence] = tags_dict
        for i in ((temp_tags)):
            tags_dict[i[1]].append(i[0])
    return tags_dict_sentences

## **Generate Blanks**

In [ ]:
def replaceblank(word, sentence):
    temp = re.compile(re.escape(word), re.IGNORECASE)
    return temp.sub('__________________', sentence),sentence

In [ ]:
def removeWord(sentence, tags_dict):
    words = None
    if 'NNP' in tags_dict:
        words = tags_dict['NNP']
    elif 'NN' in tags_dict:
        words = tags_dict['NN']
    else:
        print("NN and NNP not found")
        return (None, sentence, None)
    if len(words) > 0:
        word = random.choice(words)
        replaced = replaceblank(word, sentence)
        return (word, sentence, replaced)
    else:
        print("words are empty")
        return (None, sentence, None),sentence

In [ ]:
def create_blanks(paragraph,num_of_blanks):
    global tags_dict_sentences
    tags_dict_sentences = clean_and_tag(paragraph)
    # these many blanks are possible, as they can't be more than number of sentences
    global possib_blanks
    possib_blanks = len(list(tags_dict_sentences.keys()))
    print(possib_blanks)
    if possib_blanks> num_of_blanks:
        # randomly shuffle the sentences
        rand_sents = (list(tags_dict_sentences.keys()))
        random.shuffle(rand_sents)
        
        #store the blanks in an array
        global blanks_arr
        blanks_arr = []
        # number of prepared blanks and dummy variable to iterate through rand_sents
        prepared_blanks = 0
        i = 0
        while prepared_blanks<num_of_blanks and (i<len(rand_sents)):
            curr_sent_tags = tags_dict_sentences[rand_sents[i]]
            (word, sentence, replaced) = removeWord(rand_sents[i], tags_dict_sentences[rand_sents[i]])
            if replaced is not None:
                blanks_arr.append([replaced,word])
                i+=1
                prepared_blanks+=1
            else:
                i+=1
        if prepared_blanks<num_of_blanks:
            print("sorry, couldn't form more than {} blanks".format(prepared_blanks))
        return blanks_arr
    # the same as above but asking for input
    else:
        try:
            num_of_blanks  = int(input('''number of blanks you want to create are more
            than number of sentences, please give a number less than {}.  
            if you want to quit, just press any key other than integer and enter\n'''.format(
            possib_blanks)))
            if num_of_blanks<possib_blanks:
                flag = 0
                rand_sents = (list(tags_dict_sentences.keys()))
                random.shuffle(rand_sents)
                blanks_arr = []
                prepared_blanks = 0
                i = 0
                while prepared_blanks<num_of_blanks and (i<len(rand_sents)):
                    curr_sent_tags = tags_dict_sentences[rand_sents[i]]
                    (word, sentence, replaced) = removeWord(rand_sents[i], tags_dict_sentences[rand_sents[i]])
                    if replaced is not None:
                        blanks_arr.append([replaced,word])
                        i+=1
                        prepared_blanks+=1
                    else:
                        i+=1
                if prepared_blanks<num_of_blanks:
                    print("sorry, couldn't form more than {} blanks".format(prepared_blanks))
                return blanks_arr
        except:
            print('quittin :(')
            return None
        

**Take Input From Users locally**

In [ ]:
generate_ques=int(number_of_question)

## **Total Generated Blanks**

In [ ]:
blanks = create_blanks(words_to_sentense,generate_ques)

72
NN and NNP not found


## **Generaed Blanks From Text**

In [ ]:
generated_blanks=[]
all_blanks = create_blanks(words_to_sentense,(possib_blanks-1))
for blank in all_blanks:
    generated_blanks.append(blank)

72
NN and NNP not found
NN and NNP not found
NN and NNP not found
NN and NNP not found
NN and NNP not found
sorry, couldn't form more than 67 blanks


In [ ]:
generated_answer = [{'question': l[0][0], 'context':l[0][1],'answer': l[1]} for l in generated_blanks]

## **Remove Duplicate Blanks**

In [ ]:
new_questions = []
for i in generated_answer:
    if i not in new_questions:
        new_questions.append(i)

In [ ]:
scoring = []
for i in new_questions:
  #e=list(i.keys())[2]
   e=i.get("context")
   scoring.append(e)
   

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = words_to_sentense
de=[]
for i in scoring:
  d = question_answerer(question=i, context=context)
  e=d.get("score")
  e_updated=1-float(e)
  accuracy_score="%.2f" % round(e_updated*100, 6)
  de.append(accuracy_score)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
for option_index, option in enumerate(new_questions):
    option["rank"] = float(de[option_index])

In [ ]:
final_generated_questions = sorted(new_questions, key=lambda i: i['rank'], reverse=True)

In [ ]:
wrong_blank = []
right_blank = []
for i in final_generated_questions:
    quest = i.get("question")
    if quest[0]=='_' or quest[-2]=='_':
        wrong_blank.append(i)
    else:
        right_blank.append(i)

## Question seperation based on user input

In [ ]:
reduced_wrong_blank = []
for i in wrong_blank:
  rank = (i['rank'] - 20.00)
  if i['rank']  <= 20:
    reduced_wrong_blank.append({'question': i['question'], 'answer':i['answer'], 'context': i['context'], 'rank' : i['rank']})
  else:
    reduced_wrong_blank.append({'question': i['question'], 'answer':i['answer'], 'context': i['context'], 'rank' : rank})

In [ ]:
final_required_data = right_blank + reduced_wrong_blank
final_data = sorted(final_required_data, key=lambda i: i['rank'],reverse=True)

In [ ]:
if generate_ques > len(new_questions) or generate_ques == 0:
    print("please give value between 1 to {}".format(len(new_questions)))
else:
    middle_index = generate_ques
    user_required_questions = final_data[:middle_index]
    other_questions = final_data[middle_index:]

    
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

In [ ]:
end = time.time()

In [ ]:
dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1

## **Final Output , User Validation & Post Generated Questions**

In [ ]:
url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
headers = {'Content-Type':'application/json','Accept':'application/json'}
post_array ={
    "id" : _id,
    "questions" : user_required_questions,
    "other_questions" : other_questions,
    "upload_process_time": str(total_time)
}
status = requests.post(url,headers=headers,data=json.dumps(post_array))